In [1]:
import face_recognition
from PIL import Image, ImageDraw
import numpy as np
import os
import random
import json
import shutil

In [2]:


def organize_images(image_path, directory="assets/image_data/archive/push", paging_file="assets/image_data/archive/push/image_index.json"):
    """
    Organizes images into folders based on similarity to existing images,
    using face encodings for comparison. Handles multiple faces per image.

    Args:
        image_path (str): Path to the new image to be organized.
        directory (str, optional): Base directory for storing images. 
                                     Defaults to "image_store".
        paging_file (str, optional): Name of the JSON file to store 
                                      folder-encoding mappings. 
                                      Defaults to "image_index.json".
    """

    if not os.path.exists(directory):
        os.makedirs(directory)

    try:
        with open(paging_file, 'r') as f:
            folder_encodings = json.load(f)
    except FileNotFoundError:
        folder_encodings = {}

    try:
        new_image = face_recognition.load_image_file(image_path)
        new_encodings = face_recognition.face_encodings(new_image)  # Get all encodings
    except IndexError:
        print(f"Warning: No face detected in {image_path}. Skipping.")
        return

    for new_encoding in new_encodings:  # Iterate through each detected face
        best_match_folder = None
        best_match_distance = float('inf')

        # Check if the face matches any existing folder
        for folder, encoded_image in folder_encodings.items():
            distance = face_recognition.face_distance([encoded_image], new_encoding)[0]
            if distance < best_match_distance:
                best_match_distance = distance
                best_match_folder = folder

        # If a match is found, copy the image to the corresponding folder
        if best_match_folder and best_match_distance < 0.6:  # Adjust threshold as needed
            folder_path = os.path.join(directory, best_match_folder)
        else:
            # Create a new folder for a new face
            while True:
                new_folder = str(random.randint(10000, 99999))
                folder_path = os.path.join(directory, new_folder)
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                    folder_encodings[new_folder] = new_encoding.tolist()
                    break

        # Copy the image to the folder
        image_filename = os.path.basename(image_path)
        new_image_path = os.path.join(folder_path, image_filename)
        shutil.copy2(image_path, new_image_path)  # Copy the image to the matching/new folder

    # Save the updated encodings back to the JSON file
    with open(paging_file, 'w') as f:
        json.dump(folder_encodings, f)

folder_to_pick_from = "assets/image_data/archive/pull"
folder_to_compare_from = "assets/image_data/archive/push"

for item_name in os.listdir(folder_to_pick_from):
    # Example usage:
    item_path = os.path.join(folder_to_pick_from, item_name)
    organize_images(item_path) 

In [ ]:
# Load an image with an unknown face
unknown_image = face_recognition.load_image_file("two_people.jpg")

# Find all the faces and face encodings in the unknown image
face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

In [ ]:
# Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
pil_image = Image.fromarray(unknown_image)
# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)

In [ ]:
# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

    name = "Unknown"

    # If a match was found in known_face_encodings, just use the first one.
    # if True in matches:
    #     first_match_index = matches.index(True)
    #     name = known_face_names[first_match_index]

    # Or instead, use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

    # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


In [ ]:
# Remove the drawing library from memory as per the Pillow docs
del draw

In [ ]:
# Display the resulting image
pil_image.show()